# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/70d553a3f7a666707fa83df275ae2272a050574299736d2087c7c5b7.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Chicago, Illinois, United States**, and the stations the data comes from are shown on the map below.

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'70d553a3f7a666707fa83df275ae2272a050574299736d2087c7c5b7')

In [ ]:
df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/70d553a3f7a666707fa83df275ae2272a050574299736d2087c7c5b7.csv')
%matplotlib notebook
import matplotlib as mpl
import numpy as np

In [ ]:
df = df[~df.Date.str.endswith('02-29')]

In [ ]:
df['Data_Value'] = df['Data_Value'] * .1
df['Year'] = df.Date.str[:4]
df['Day'] = df.Date.str[5:]

In [ ]:
maxtemp = df[(df['Element']=='TMAX') & (df['Year'] != '2015')].groupby(by='Day').aggregate({'Data_Value':np.max})
mintemp = df[(df['Element']=='TMIN') & (df['Year'] != '2015')].groupby(by='Day').aggregate({'Data_Value':np.min})

In [ ]:
maxtemp15 = df[(df['Element']=='TMAX') & (df['Year'] == '2015')].groupby(by='Day').aggregate({'Data_Value':np.max})
mintemp15 = df[(df['Element']=='TMIN') & (df['Year'] == '2015')].groupby(by='Day').aggregate({'Data_Value':np.min})

In [ ]:
max_break = np.where(maxtemp15.Data_Value > maxtemp.Data_Value)
min_break = np.where(mintemp15.Data_Value < mintemp.Data_Value)

In [ ]:
plt.figure(figsize=(12,6))
plt.gca().axis([0, 365, -30,40])
plt.plot(maxtemp.values, 'r', alpha=0.30)
plt.plot(mintemp.values, 'b', alpha=0.3)
plt.scatter(max_break, maxtemp15.iloc[max_break], s=20, c='black', label = 'Record High 2015')
plt.scatter(min_break, mintemp15.iloc[min_break], s=20, c='black', label = 'Record Low 2015')
plt.gca().fill_between(range(len(mintemp)),mintemp.Data_Value, maxtemp.Data_Value,facecolor='blue', alpha=0.10)
plt.legend(['Record High 2005-2014', 'Record Low 2005-2014','Record breaking days in 2015'], frameon=False)
plt.ylabel('Degrees Celsius')
plt.xlabel('Day of the Year')
plt.title('Chicago Record Temperatures')
plt.subplots_adjust(bottom=0.10)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.show()
image = 'image.png'
plt.savefig(image)
from IPython.display import HTML
HTML('<a href ="{0}" download>Click here to download {0}</a>'.format(image))